In [104]:
# Import standard libraries
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import random
import numpy as np
import yfinance as yf # ticker data 
from pandas_datareader import data as pdr # market proxy and treasury rates data
import datetime as dt
from numpy import nan
from time import sleep
import concurrent.futures
from functools import reduce
from scipy.stats import norm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import scipy 
import time

In [105]:
# Fetch historical price data for the desired asset using our data
company_dataset = pd.read_csv("Data/CRSP_stocks_clean.csv")

In [106]:
company_dataset

,Unnamed: 0,date,TICKER,CUSIP,PRC,RET,VOL,BID,ASK,SHROUT
0,1718,2000-01-03,AIR,36110,17.5625,-0.020906,109000.0,17.375,17.750,27401.0
1,10251,2000-01-04,AIR,36110,17.6250,0.003559,51700.0,17.375,17.875,27401.0
2,18784,2000-01-05,AIR,36110,17.8125,0.010638,70400.0,17.500,18.000,27401.0
3,27320,2000-01-06,AIR,36110,17.9375,0.007018,60100.0,17.750,18.000,27401.0
4,35852,2000-01-07,AIR,36110,18.5625,0.034843,148000.0,18.125,18.750,27401.0
...,...,...,...,...,...,...,...,...,...,...
8109235,42531451,2022-12-23,TK,Y8564W10,4.5800,-0.002179,422897.0,4.580,4.590,101872.0
8109236,42540998,2022-12-27,TK,Y8564W10,4.5700,-0.002183,664421.0,4.570,4.580,101872.0
8109237,42550538,2022-12-28,TK,Y8564W10,4.3100,-0.056893,1103593.0,4.310,4.320,101872.0
8109238,42560071,2022-12-29,TK,Y8564W10,4.4300,0.027842,764632.0,4.420,4.430,101872.0


In [107]:
# Group the data by the ticker symbol
grouped_data = company_dataset.groupby('TICKER')
print(company_dataset[company_dataset['TICKER'] == 'AAPL'])

         Unnamed: 0        date TICKER     CUSIP        PRC       RET  \
2101151         578  2000-01-03   AAPL  03783310  111.93750  0.088754   
2101152        9112  2000-01-04   AAPL  03783310  102.50000 -0.084310   
2101153       17645  2000-01-05   AAPL  03783310  104.00000  0.014634   
2101154       26181  2000-01-06   AAPL  03783310   95.00000 -0.086538   
2101155       34714  2000-01-07   AAPL  03783310   99.50000  0.047368   
...             ...         ...    ...       ...        ...       ...   
2106933    42525026  2022-12-23   AAPL  03783310  131.86000 -0.002798   
2106934    42534569  2022-12-27   AAPL  03783310  130.03000 -0.013878   
2106935    42544117  2022-12-28   AAPL  03783310  126.04000 -0.030685   
2106936    42553654  2022-12-29   AAPL  03783310  129.61000  0.028324   
2106937    42563190  2022-12-30   AAPL  03783310  129.92999  0.002469   

                VOL        BID        ASK      SHROUT  
2101151   4833736.0  111.87500  111.93750    161159.0  
2101152   4

In [108]:
# Function to calculate Value-At-Risk and Expected Shortfall
def calculate_VaR(returns, alpha=0.05):
    return np.percentile(returns, 100 * alpha)

def calculate_ES(returns, alpha=0.05):
    VaR = np.percentile(returns, 100 * alpha)
    returns_below_var = returns[returns <= VaR]
    return -np.mean(returns_below_var)

In [109]:
# Calculate the Expected Shortfall for each ticker
expected_shortfalls = {}
for ticker, group in grouped_data:
    returns = group['RET'].dropna()  # Assuming 'ret' column contains daily returns
    es = calculate_ES(returns)
    expected_shortfalls[ticker] = es

In [124]:
# Print the results
print(f"Expected Shortfall (ES)")
for ticker, es in expected_shortfalls.items():
    print(f"{ticker}: {es}")

Expected Shortfall (ES)
A: 0.06029848965517237
AA: 0.0621230938967136
AAIC: 0.03985692857142858
AAME: 0.10990485517241377
AAN: 0.05671442176870747
AAON: 0.05560636551724135
AAPL: 0.05569163103448274
AAS: 0.06657657142857143
AB: 0.0578976666666667
ABC: 0.039764382899628234
ABCB: 0.06616567931034482
ABEV: 0.04969760344827588
ABFS: 0.07057203314917128
ABIO: 0.11198307909604516
ABM: 0.048244913793103444
ABV: 0.07257259880239522
ABX: 0.0585931673640167
AC: 0.044387102564102575
ACE: 0.04620909625668451
ACGL: 0.03709550877192982
ACHV: 0.12221356521739128
ACIW: 0.05766965641025645
ACL: 0.05243756250000001
ACP: 0.040023020618556705
ACTY: 0.1598054333333333
ACXM: 0.06536631779661019
ACY: 0.10575162500000006
ADBE: 0.06257369655172418
ADC: 0.048905562068965486
ADI: 0.05881213448275861
ADM: 0.04494321724137933
ADP: 0.03656642446043167
ADSK: 0.06012556551724135
ADTN: 0.06512989310344827
AEE: 0.031963975862068957
AEF: 0.0346574
AEG: 0.07055925517241386
AEHR: 0.10912819310344833
AEIS: 0.08009015862068